In [1]:
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import Input, layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import random as rn
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile

import sounddevice as sd

import pandas as pd
import tensorflow as tf
import glob

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

import IPython.display as ipd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import style
import seaborn as sns

%matplotlib inline
sns.set(style='whitegrid', color_codes=True)

# Pre processing

# Deep learning

In [2]:
seed = 100
np.random.seed(seed)
rn.seed(seed)
tf.set_random_seed(seed)

In [3]:
labels = pd.read_csv('train_labels.csv')
sample_solution = pd.read_csv('sample_submission.csv')
train = np.load('train_data.npy')
test = np.load('test_data.npy')

In [4]:
print(train[1])
print(test[1])

Y = labels['Label']

print(Y)

[0.00110629 0.00221578 0.00401346 ... 0.02375466 0.01968718 0.01686463]
[-4.16994095e-03 -3.57834622e-03 -2.97976262e-03 ... -9.80041368e-05
 -1.81882875e-03 -1.71524123e-03]
0       0
1       0
2       0
3       1
4       0
       ..
3994    0
3995    0
3996    0
3997    0
3998    0
Name: Label, Length: 3999, dtype: int64


In [5]:
# sampling frequency
fs = 44100
#sd.play(train[6], fs)
sd.play(test[34], fs)

In [9]:
# train: we see 3999 records of data with sampling frequency of 44100
# test: we see 3997 records of data with sampling frequency of 44100

drop_labels = labels.drop('Id',1)
Y_train = to_categorical(drop_labels ,2)

x_train, x_test, y_train, y_test = train_test_split(
    train, Y_train, test_size=0.1, random_state=seed)

x_train = x_train.reshape(-1,44100,1)
x_test = x_test.reshape(-1,44100,1)
# y_train = y_train.reshape(-1,-1,1)
# y_test = y_test.reshape(-1,-1,1)
print("TRAIN SHAPE")
print(x_train.shape, y_train.shape)
print("TEST SHAPE")
print(x_test.shape, y_test.shape)

TRAIN SHAPE
(3599, 44100, 1) (3599, 2)
TEST SHAPE
(400, 44100, 1) (400, 2)


In [10]:
n_timesteps, n_features, n_outputs = x_train.shape[0], x_train.shape[1], y_train.shape[0]
print('# of samples / timestamps:', n_timesteps)
print('frequency    / features  :', n_features)
print('# of outputs / outputs   :', n_outputs)

# of samples / timestamps: 3599
frequency    / features  : 44100
# of outputs / outputs   : 3599


In [13]:
# create convolution neural network
model = Sequential()
model.add(Conv1D(64, 3, activation='tanh', batch_size=44100))
model.add(MaxPooling1D(3))
model.add(Conv1D(32, 3, activation='tanh'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))

In [ ]:
model.summary()

In [14]:
epochs, batch_size = 20, 61
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_data=(x_test, y_test))

Train on 3599 samples, validate on 400 samples
Epoch 1/20
3599/3599 [==============================] - 8s 2ms/sample - loss: 0.3005 - acc: 0.8700 - val_loss: 0.2136 - val_acc: 0.8875
Epoch 2/20
3599/3599 [==============================] - 8s 2ms/sample - loss: 0.2021 - acc: 0.8805 - val_loss: 0.2135 - val_acc: 0.8900
Epoch 3/20
3599/3599 [==============================] - 8s 2ms/sample - loss: 0.1748 - acc: 0.8886 - val_loss: 0.2132 - val_acc: 0.9000
Epoch 4/20
3599/3599 [==============================] - 8s 2ms/sample - loss: 0.1630 - acc: 0.8994 - val_loss: 0.2227 - val_acc: 0.8925
Epoch 5/20
3599/3599 [==============================] - 8s 2ms/sample - loss: 0.2666 - acc: 0.8794 - val_loss: 0.6827 - val_acc: 0.8875
Epoch 6/20
3599/3599 [==============================] - 8s 2ms/sample - loss: 0.6818 - acc: 0.8769 - val_loss: 0.6827 - val_acc: 0.8875
Epoch 7/20
3599/3599 [==============================] - 7s 2ms/sample - loss: 0.6818 - acc: 0.8769 - val_loss: 0.6827 - val_acc: 0.8875
E

In [15]:
score = model.evaluate(x_test, y_test, batch_size=128)

400/400 [==============================] - 0s 1ms/sample - loss: 0.6931 - acc: 0.8875


In [26]:
final_predictions = model.predict_classes(test)

In [28]:
sample_solution['Label'] = final_predictions

In [29]:
sample_solution.to_csv('audio_binary_classification_predict.csv', index=False)